# Using the Gemini API
This notebook goes over all of the necessary steps to get started with the Gemini API starting from scratch.
## Step 1: Make sure you're signed into your ASU Google account


## Step 2: Visit the [Google AI Dev](https://ai.google.dev/) website.
There are many differnt guides and documentation pages listed here, its a good resource to have.

## Step 3: Accessing the API on Google Cloud
### Make the Cloud Project
1. Click on the "Get API Key in Google AI Studio" button on the Google AI Dev website. It should link [here](https://aistudio.google.com/app/apikey). 
	
	Note that when I tried clicking on the "Create API key" button it didn't work and gave the following error. 
	```
	code: 7
	message: "Permission \'resourcemanager.projects.create\' denied on parent resource \'organizations/247721031950\'."
	details {
		type_url: "type.googleapis.com/google.rpc.ResourceInfo"
		value: "\n\aproject\022#projects/gen-lang-client-0325843885"
	}
	```
	
	This seems to be something blocked by ASU, so we'll have to use this the hard way.
	
2. Find the side-bar button labeled "Gemini API for Enterprise" and click on it.
3. At the top left there should be a button to create a new project. Click on it.
4. Fill out the name for your project, mine is "LLMs", but it can be anything. Make sure the organization is "asu.edu". For the location, click browse and you should see "Unmanaged". Then click "Create".

	Note that the default project which should show up fails to create. I believe this is because ASU blocks it in the default directory, which is why we need "Unmanaged".

### Enable Billing.
There should be a button that pops up to do this. I can't trigger it again since I already have billing enabled, but this should be pretty straightforward.

You can turn off autorenewing to avoid any charges after your free credits are used up.

### Enable the API
There should be a popup to enable the Vertex AI API. Click on the "Enable" button.

If it doesn't show up immediately, it should show up when you're clicking around the model garden mentioned in the next section.

### Testing in Colab Enterprise
There are default notebooks for interacting with Gemini on the Google Cloud website.
1. Go to the [Vertex AI](https://console.cloud.google.com/vertex-ai) website. It should automatically open the project you just made, however make sure you are in the correct one if you have multiple.
2. From here, you can click "Model Garden" on the side bar, the icon should be a couple of leaves.
3. Under the "Foundation Models" section, you have the choice from what they have available.

	I chose [Gemini 1.0 Pro](https://console.cloud.google.com/vertex-ai/publishers/google/model-garden/gemini-pro) since it seems like it costs less than the other options.

4. Click on the "Open Notebook" button. This will open a Colab Enterprise session with a notebook showcasing different API calls.

	Note that the code you run here eats into your credits.

## Step 4: Accessing the API Locally
### Setting up local authentication.
Detailed instructions can be found [here](https://cloud.google.com/iam/docs/keys-create-delete#creating), and I outlined them in this notebook.

#### Install the CLI
Visit the Google Cloud SDK [installation website](https://cloud.google.com/sdk/docs/install) and follow the instructions for your OS.

Run this command in your a new terminal window and make sure you select the project you created here.
```bash
gcloud init
```
Log in. This is not necessary on a cloud shell or if you are already logged into your ASU account.
```bash
gcloud auth application-default login
```

### Example code to use the API
This follows the same steps as in the provided notebook, but locally.	

Authentication for Colab; included in case this notebook is imported into Colab. Note that you would need to be signed into your ASU account.

In [1]:
# Additional authentication is required for Google Colab
import sys
if "google.colab" in sys.modules:
	# Authenticate user to Google Cloud
	from google.colab import auth
	print("Authenticating user to Google Cloud...")
	auth.authenticate_user()

In [2]:
%pip install --upgrade --user -q google-cloud-aiplatform # Includes Vertex AI API

Note: you may need to restart the kernel to use updated packages.


In [3]:
import vertexai
from vertexai.generative_models import GenerationConfig, GenerativeModel

/Users/petermousses/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Initialize Vertex AI.

In [4]:
vertexai.init(
	project='llms-441517', # You can find it by clicking on the project dropdown in the top left.
	location='us-central1'
)

Load the model. In this case it is Gemini 1.0 Pro.

In [5]:
model = GenerativeModel('gemini-1.0-pro')

Showcase of the streaming functionality. It returns text as it is generated, so it is a generator object, and therefore needs to be printed this way.

In [6]:
sky_prompt = 'Why is the sky blue?'
responses = model.generate_content(sky_prompt, stream=True)
for response in responses:
    print(response.text, end='')

The sky appears blue because of a phenomenon called Rayleigh scattering. This scattering occurs when sunlight enters the Earth's atmosphere and interacts with the gas molecules present there. 

Sunlight is composed of all the colors of the rainbow, each with a different wavelength. Blue light has a shorter wavelength than other colors like red or yellow. When sunlight encounters the gas molecules in the atmosphere, the shorter blue wavelengths are scattered more readily than the longer wavelengths. This scattered blue light reaches our eyes from all directions in the sky, making the sky appear blue. 

Here's a breakdown of the process:

1. **Sunlight enters the atmosphere:** Sunlight travels through space and reaches Earth's atmosphere.
2. **Light interacts with gas molecules:** Sunlight interacts with the molecules of nitrogen and oxygen, which are the main components of the atmosphere.
3. **Rayleigh scattering occurs:** Shorter wavelengths like blue light are scattered more powerfull

The same prompt with a custom config.

In [7]:
generation_config = GenerationConfig(
    temperature=0.9,
    top_p=1.0,
    top_k=32,
    candidate_count=1,
    max_output_tokens=8192,
)

responses = model.generate_content(
    sky_prompt,
    generation_config=generation_config,
    stream=True,
)
for response in responses:
    print(response.text, end='')

The sky appears blue because of a phenomenon called Rayleigh scattering. This scattering occurs when sunlight enters the Earth's atmosphere and interacts with the molecules of nitrogen and oxygen present there. 

Sunlight is composed of all the colors of the rainbow, each with a different wavelength. Blue light has a shorter wavelength than other colors like red or orange. When sunlight enters the atmosphere, the shorter wavelengths of blue light are scattered more readily by the nitrogen and oxygen molecules than the longer wavelengths of other colors. This scattered blue light reaches our eyes from all directions in the sky, making it appear blue.

This scattering also explains why the sky appears reddish or orange during sunrise and sunset. During these times, sunlight has to travel a longer path through the atmosphere, which causes more of the blue light to be scattered away. This leaves the longer wavelengths of red and orange light to reach our eyes, giving the sky its reddish or

Chat example.

In [8]:
chat = model.start_chat()

prompt = """My name is Ned. You are my personal assistant. My favorite movies are Lord of the Rings and Hobbit.

Suggest another movie I might like.
"""
responses = chat.send_message(prompt, stream=True)
for response in responses:
    print(response.text, end="")
print('\n','-'*25)
    
prompt = 'Are my favorite movies based on a book series?'
responses = chat.send_message(prompt, stream=True)
for response in responses:
    print(response.text, end="")
print('\n','-'*25)

print(chat.history)

Hi Ned, 

It's great to meet you! I understand you're a fan of Lord of the Rings and The Hobbit, and I think you might enjoy "The Chronicles of Narnia: The Lion, the Witch, and the Wardrobe." It's a fantasy film based on the first book in C.S. Lewis's beloved "Chronicles of Narnia" series. Like Lord of the Rings and The Hobbit, it features a group of young heroes who embark on a magical adventure in a world filled with mythical creatures and epic battles. 

Here are some reasons why you might like "The Chronicles of Narnia: The Lion, the Witch, and the Wardrobe":

* **Similar themes:** Like Lord of the Rings and The Hobbit, The Chronicles of Narnia explores themes of good versus evil, friendship, and courage.
* **Rich fantasy world:** Narnia is a wonderfully imaginative world filled with talking animals, mythical creatures, and magical landscapes.
* **Epic battles:** The film features some truly epic battles between the forces of good and evil, similar to the battles in Lord of the Rin